# Azimuthal integration of pilatus data using the python version of MATFRAIA

This notebook will use the python implementation of MATFRAIA to azimuthally integrate either a */raw/SAMPLE/\*_pilatus.h5* file in user-specified bins or an numpy array (image) and save the integrated data in a corresponding */process/azint/SAMPLE/\*_pilatus_integrated.h5* file.

**Required user-input:**  
`scans`              : List of scan indices to be integrated  
`poni_file`          : Absolute path for the .poni file  
`mask`               : Absolute path for the .npy mask file  
`radial_bins`        : Number of radial bins (default=3000)  
`azimuthal_bins`     : Number of azimuthal bins - See help(DM.integrateFile) for more  
`unit`               : "q" or "2th"  
`n_splitting`        : Number of sub-pixel splitting used. The actual number of sub-pixels is N^2. (default=15)  
`polarization_factor`: Polarization factor (default=0.999997)  


More information about the integration can be found here: [Azimuthal binning](https://wiki.maxiv.lu.se/index.php?title=DanMAX:_Azint_pipeline#Azimuthal_binning)

In [ ]:
%matplotlib widget
import DanMAX as DM
import numpy as np
import matplotlib.pyplot as plt
import azint

#### Setup integration configurations

In [ ]:
poni = '/data/visitors/danmax/PROPOSAL/VISIT/process/PONI_FILE.poni'
mask = '/data/visitors/danmax/PROPOSAL/VISIT/process/MASK_FILE.npy'

config = {'poni'   : poni,
          'mask'        : mask,
          'radial_bins' : 3000,
          'azimuth_bins': None,
          'n_splitting' : 15,
          'polarization_factor': 0.999997,
          'unit'        : '2th',
         }

#### Integrate files
This cell showcase how to iteratively integrate .h5 files from a list of scan numbers

In [ ]:
# list of scan numbers
scans = [1234,1235,1236]

# set integrator objects to None for the first iteration
ai_1d, ai_2d = None, None
for scan in scans:
    # find file name from scan number
    fname = DM.findScan(scan)
    print(DM.getScan_id(fname))
    # generate file name for azint data
    aname = fname.replace('/raw/', '/process/azint/').split('.')[0]+'_pilatus_integrated.h5'
    # integrate
    data, meta, ai_1d, ai_2d = DM.integration.integrateFile(fname,config, ai_1d=ai_1d, ai_2d=ai_2d,im_corr=None)
    # save to .h5 file
    DM.integration.writeAzintFile(aname,data,meta=meta)

#### Integrate images
This cell showcase how to integrate images from a single .h5 file. Useful for implementing 2D corrections prior to integration.

In [ ]:
scan = 1234
# find file name from scan number
fname = DM.findScan(scan)
# generate file name for azint data
aname = fname.replace('/raw/', '/process/azint/').split('.')[0]+'_pilatus_integrated.h5'

# HDF5 dataset entry path
dset_name = '/entry/instrument/pilatus/data'
# read images - Memory heavy!
with h5.File(fname,'r') as f:
    im = f[dset_name][:]
    
if False:
    t0 = .2 # sample thickness in mm
    mu =  20. * 10**-1  # Absorption coefficient in cm-1
    A = DM.texture.absorptionCorrection2D(poni, t0, mu, omega=0, chi=0, phi=0, normalize=True)
    im *= A
# integrate
data, meta, ai_1d, ai_2d = DM.integration.integrateImages(im,config, ai_1d=ai_1d, ai_2d=ai_2d)
# save to .h5 file
DM.integration.writeAzintFile(aname,data,meta=meta)